In [1]:
import os
import sys
import json
import pandas as pd
import qgrid
from keybert import KeyBERT
from bertopic import BERTopic

In [44]:
cwd = !pwd
sys.path.append(os.path.split(cwd[0])[0])

In [3]:
from fantastic_amazon import amazon_scraper, sheets_connector

In [270]:
PN_FOR_ANAYLSIS = 'CATRIPLE4KDOCKPDUS'

## Connect to Google Sheets 

In [260]:
SERVICE_FILE = "/home/max/fantastic-amazon-c1ba8f85a23b.json"
SHEET_KEY = "1M8FT1qtKr5VgUAk-7IvQdX8Hp2Sn13uZLL3tq2o7YOU"
SHEET_NAME = "PRICES"

In [261]:
try:
    gs
except NameError:
    gs = sheets_connector.GoogleSheets(sheet_key=SHEET_KEY, sheet_name=SHEET_NAME, service_file=SERVICE_FILE) 

In [262]:
PNs = gs.w.range("B3:B30")
url_dict = {} 
for PN in PNs:
    url_row = 'H' + str(PN[0].row) + ':J' + str(PN[0].row) 
    urls = gs.get_urls(url_row)
    url_dict[PN[0].value] = urls

In [263]:
list(url_dict.keys())

['C31DUAL4KDP',
 'C31DUAL4KHDMI',
 'C31DUALDPDOCKPD',
 'C31DUALDPDOCKPD100WUS',
 'C31DUALDPDOCKPD65WUS',
 'C31FLATDOCKPDPRO',
 'C31FLATPRO112WUS',
 'C31NANODOCKLANPD',
 'C31NANODOCKPD',
 'C31NANODOCKPROPD',
 'C31NANODOCKVGAPD',
 'C31QUATTRODOCKPDUS',
 'C31TRIPLEDOCKPDUS',
 'CADUA4KDOCKPDLUS',
 'CADUAL4KDOCKPDUS',
 'CADUAL4KDOCKUS',
 'CATRIPLE4KDOCKPDUS',
 'TB3CDUALDPDOCKPDUS',
 'TB3DUAL4KDP',
 'TB3DUAL4KHDMI',
 'TB3HDMIDOCKUS',
 'TB3TRAVELDOCKPD',
 'TB3TRIPLEDOCKPDUS',
 'U3DUAL4KDP',
 'U3DUAL4KHDMI',
 'U3DUALHDMIDOCKUS',
 'U3HDMIDVIDOCKUS',
 'VESADOCK1']

## Read in the Product Page

In [271]:
with open(os.path.join('..', 'data', 'product_info.json'), 'r') as f:
    prod = json.load(f)

In [272]:
# prod['C31DUAL4KDP']['https://www.amazon.com/StarTech-com-2-Port-USB-C-DisplayPort-MST/dp/B01KIMP28Y/ref=sr_1_5?dchild=1&keywords=USB-C+to+Dual+DisplayPort+Video+Adapter&qid=1621983565&sr=8-5']'https://www.amazon.com/StarTech-com-2-Port-USB-C-DisplayPort-MST/dp/B01KIMP28Y/ref=sr_1_5?dchild=1&keywords=USB-C+to+Dual+DisplayPort+Video+Adapter&qid=1621983565&sr=8-5'
titles = [prod[PN_FOR_ANAYLSIS][l]['title'] for l in url_dict[PN_FOR_ANAYLSIS] if l]
[print (title + "\n") for title in titles]

StarTech.com USB-C & USB-A Dock - Hybrid Universal Triple Monitor Laptop Docking Station w/DisplayPort & HDMI 4K 60Hz - 85W Power Delivery, 6X USB Hub, GbE, Audio - USB 3.1 Gen 2 10Gbps (DK31C3HDPD)

ICY BOX Type-C or Type-A Docking Station with Triple Video Output for Notebook/Laptop/PC and 60W Power Delivery (3X HDMI, 6X USB Ports, 2X DisplayPorts, Gigabit LAN and Audio)



[None, None]

In [273]:
url_dict[PN_FOR_ANAYLSIS]

['https://www.amazon.com/StarTech-com-USB-C-USB-Dock-DisplayPort/dp/B08J4HD9YM/ref=sr_1_6?dchild=1&keywords=universal+docking+station+2x+hdmi+2x+displayport&qid=1621987828&s=electronics&sr=1-6',
 'https://www.amazon.com/Docking-Station-Notebook-Delivery-DisplayPorts/dp/B07VQTHK1M/ref=sr_1_38?dchild=1&keywords=universal+docking+station+2x+hdmi+2x+displayport+power+delivery&qid=1621988191&s=electronics&sr=1-38',
 '']

## Pull First Page of Search Terms

In [278]:
one_search = amazon_scraper.AmazonScrapper().scrape_search("amazon.com", search_terms="Thunderbolt 3 Dual Docking Station", headless=True)
other_search = amazon_scraper.AmazonScrapper().scrape_search("amazon.com", search_terms="Thunderbolt 3 8k hub", headless=True)

In [279]:
one_search['suggestions']

['thunderbolt 3 docking station dual monitor',
 'in Electronics',
 'startech thunderbolt 3 dual-4k docking station',
 'monoprice thunderbolt 3 dual displayport docking station']

In [280]:
other_search['suggestions']

['thunderbolt 3 hub 8k']

In [281]:
cutoff_price = 100

In [282]:
df = pd.DataFrame.from_records([value for key, value in one_search.items() if key != 'suggestions' if value['price'] > cutoff_price], columns=list(one_search['2'].keys()))
o_df = pd.DataFrame.from_records([value for key, value in other_search.items() if key != 'suggestions' if value['price'] > cutoff_price], columns=list(other_search['2'].keys()))

In [283]:
c_df = pd.concat([df,o_df]).drop_duplicates(['asin'], keep=False, )
d_df = pd.concat([df, o_df]).loc[pd.concat([df, o_df]).duplicated(subset=['asin'])]
from IPython.display import HTML
HTML(c_df.to_html())

,asin,ad,amazon_choice,title,price,coupon,review_num
0,B08P5LQ1XQ,True,False,"Thunderbolt 3 Docking Station with Built-in TB3 Cable for Mac and Windows. 60W Upstream Power Delivery Charging. Dual 4K Display, 8K Single Display and up to 40Gbps Transfer speeds, Black",153.37,False,1
1,B08DDH5CPW,True,False,"Plugable USB C Triple Display Docking Station with Laptop Charging, Thunderbolt 3 or USB C Dock Compatible with Specific Windows and Mac Systems (3x HDMI, 6x USB Ports, 60W USB PD)",159.00,False,197
3,B0785DQLZ7,False,False,Kensington SD5200T Thunderbolt 3 Docking Station - 85W PD - Dual Monitor 4k for Mac and PC (K38300NA),162.18,False,87
4,B08HR3T837,False,False,"Plugable 14-in-1 USB-C and Thunderbolt 3 Dock - Compatible with Mac and Windows, 96W Laptop Charging, 2x HDMI 2.0 and DisplayPort, 7x USB ports, Ethernet, Audio, SD/MicroSD",249.00,False,355
5,B086H3GQNS,False,False,"Amazon Basics 11-in-1 Thunderbolt 3 Docking Station - 2x DisplayPort 1.4, 4xUSB ports, SD, microSD, Ethernet, Audio, 120W power and Thunderbolt 3 Cable for iMac, iPad Pro, PC, Laptops",161.49,False,5
6,B07XQKYSK3,True,False,"SIIG Thunderbolt 3 or USB C Dual 4K Display Docking Station with 60W Laptop Charging - Titan Ridge (1x USB 3.0, 2x USB 3.1, 2x DP 1.2, 2x USB-C, Ethernet) for Windows PC & MacBook Pro JU-DK0C11-S1",233.88,False,2
7,B08KHP5MV8,False,False,"Plugable Thunderbolt 3 and USB C Docking Station with 96W Charging - Compatible with Mac and Windows Laptops, DisplayPort and HDMI, 2X USB-C, 3X USB 3.0, Gigabit Ethernet, Audio Jack - Horizontal",249.00,False,85
8,B073JDZGKK,False,False,"StarTech.com Thunderbolt 3 Dock - Dual Monitor 4K 60Hz Laptop Docking Station with DisplayPort - 85W Power Delivery - 3-Port USB 3.0 Hub, Ethernet, Audio - TB3 Dock - Windows & Mac (TB3DK2DPPD)",252.95,False,93
9,B08K3XZB43,False,False,"Intpw USB C Laptop Docking Station Dual Monitor, Thunderbolt 3 Dock w/ 60W Power Delivery, MacBook Pro Docking Station w/ 2 HDMI, DP, 6 USB Ports, Ethernet, USB-C Gen2, SD/TF Card Reader",149.99,True,102
10,B07F94KD1L,False,False,"StarTech.com Thunderbolt 3 Dock - Dual 4K 60Hz Monitor TB3 Laptop Docking Station with DisplayPort, HDMI & 1080p VGA - 85W Power Delivery & Charging - 2x USB-A, Ethernet - Mac & Windows (TB3DK2DHV)",264.99,False,349


In [284]:
from IPython.display import HTML
HTML(d_df.to_html())

,asin,ad,amazon_choice,title,price,coupon,review_num
3,B07VL5XLBS,True,False,"Kensington Thunderbolt 3 Docking Station, SD Card Reader, 60W PD and Dual 4K HDMI for Mac and Windows",249.63,False,329


In [285]:
doc = "\n".join(d_df['title'])
kw_model = KeyBERT('distilbert-base-nli-mean-tokens')
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words='english')

In [286]:
keywords

[('4k hdmi mac', 0.763),
 ('hdmi mac windows', 0.6642),
 ('dual 4k hdmi', 0.6515),
 ('hdmi mac', 0.6361),
 ('4k hdmi', 0.6064)]

In [287]:
doc = "\n".join(c_df['title'])
kw_model = KeyBERT('distilbert-base-nli-mean-tokens')
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words='english')

In [288]:
keywords

[('hdmi 40gb usb', 0.6912),
 ('macbook pro docking', 0.686),
 ('3x usb gigabit', 0.6838),
 ('hdmi 6x usb', 0.6805),
 ('display 4x usb', 0.6646)]